## Two - Way Fixed Effects Model
- 이전에 공부했던 Propensity Score, Matching 등의 분석 방법은 교란변수를 통제해야 되는 $(Y_0, Y_1) \perp T | X$ 가정에 의존하지만, 관측할 수 없는 변수가 존재하기 때문에 이전 분석 방법들을 활용했을 때 Bias 문제가 발생하게 된다.

- 또한 관측할 수 없는 변수가 존재할 때, 도구변수를 활용하여 분석할 수 있지만 좋은 도구변수를 구하기 위해선 많은 창의성을 요구하기 때문에 분석하기가 쉽지 않다.
  
- 따라서 Two - Way Fixed Effects Model을 활용하는 것이 위 방법론들의(Propensity Score, Matching, Instrumental Variables etc...) 문제들을 보완할 수 있는 하나의 대안이 될 수 있다.

### The Purpose of Two - Way Fixed Effects Model
- 단위(Unit) 와 시간(Time) 변수들이 관측할 수 없는 변수들을 대신하여 통제하는 분석 방법이다.

### Panel Data
- Unit Fixed Effects 와 Two - Way Fixed Effects는 Panel Data를 활용하여 분석한다.
  
  - Panel Data: 여러 기간에 걸쳐 동일한 단위(Unit)을 반복적으로 관찰한 데이터
  
- Panel Data를 활용한 분석은 Unit Fixed Effects, Two - Way Fixed Effects 외에도 다양한 분석 방법들이 있다.
  
  - Pooled OLS
  
  -  Seemingly Unrelated Regression(SUR)
  
  -  etc...

#### Python으로 Panel Data 구조 이해하기
- 해당 데이터를 이용하여 Unit Fixed Effects 와 Two-Way Fixed Effects Model에 대해 공부하도록 하자.

In [12]:
import pandas as pd

Panel = pd.DataFrame({
    "Unit":[1,1,1,2,2,2,3,3,3],
    "Time":[1,2,3,1,2,3,1,2,3],
    "Treatment":[0,1,1,0,1,1,0,0,0],
    "Outcome":[50,70,70,10,30,50,30,20,10]
})

Panel

,Unit,Time,Treatment,Outcome
0,1,1,0,50
1,1,2,1,70
2,1,3,1,70
3,2,1,0,10
4,2,2,1,30
5,2,3,1,50
6,3,1,0,30
7,3,2,0,20
8,3,3,0,10


### Unit Fixed Effects Model
- Unit Fixed Effects만 고려할 경우, 시간에 따라 변하지 않고 관측할 수 없는 변수들만 통제 가능하다.
  
- 따라서 동일한 Unit(혹은 Group)내 에서 시간에 따라 변하지 않는 값들은 통제(상쇄)할 수 있는 효과를 볼 수 있기 때문에 각 Unit 내에서 평균의 차이를 구하는 방법이다.

- Unit Fixed Effects Model의 식은 다음과 같다.
\begin{align}
  Y_{it} = \delta D_{it} + u_i + \varepsilon_{it}
\end{align}
- Unit Fixed Effects Model을 활용하기 위해선 4가지의 가정을 충족해야 한다.
  - Zero Conditional Mean: $E[\varepsilon_{it}\mid D_{i1}, D_{i2}, \dots, D_{iT}, u_i]=0,\quad t=1,2,\dots,T$
  
    - $Y$에 영향을 미치는 모든 요인들(Model에 포함되지 않는)이 시간에 따른 모든 관측치와 상관관계가 없다.
  
    - $Exogeneity$: 즉, $\varepsilon_{it}$는 독립 변수의 각 관측치와 상관관계가 없어야 된다.
  - $(D_{i1}, D_{i2}, \ldots, D_{it}, u_{i1}, \ldots, u_{it}), \quad i = 1, \ldots, n$ are independent and identically distributed (i.i.d.) draws from their joint distribution.
    - 이 가정은 단지 개체 간의 독립성에 관한 것이라는 점에 유의해야 된다.
  
    - 따라서 관측치가 개체 내에서 상관관계를 갖도록 허용하며, 시간이 지남에 따라 개체 내에서 자기 상관관계가 있을 수 있음을 의미한다.
  - Large outliers are unlikely
    - 큰 이상치가 없다면 관측치가 합리적인 범위에 속하기 때문에 회귀 계수 추정에 유의한 영향을 미치는 극단값을 갖지 않을 것으로 예상된다.
  
    - 따라서 이 가정은 OLS 추정치가 편향되지 않도록 보장하는 데 도움이 된다.
  - No perfect collinearity
    - 독립 변수들 사이에 정확한 선형 관계가 없을 때 성립한다. 예를 들어, 한 변수가 다른 변수의 상수 배수인 경우를 말한다.
  
    - 따라서 완전한 공선성이 있는 경우, Unit Fixed Effects Model은 OLS에 의해 추정될 수 없다.

- Unit Fixed Effects는 Unit을 각 Dummy Variable로 만들어서 선형 모델에 추가하기도 하지만, Dummy Variable 대신 demeaned 방식으로 추정하기도 한다.
  - 왜 demeaned 방식으로 추정해야 되는지는 아래에 설명되어 있다.
  
- **Dummy Variable** 방식과 **demeaned** 방식에 대해 공부해 보자.

#### Dummy Variable
- Gender, Age, Address는 일정 기간동안 시간에 따라 변화하지 않는 변수들이기 때문에 Unit Dummy variable는 세 변수들을 모두 설명할 수 있다. 따라서 세 변수들 대신 Unit Dummy Variable로 대체가능하다.
  - 세 변수들은 관측가능한 값이지만, 관측할 수 없는 변수가 있더라도 동일하게 생각할 수 있다.
  - $Ex)\;Age = 20 * UnitDummy_1 + 21 * UnitDummy_2 + 22 * UnitDummy_3$
  
- 파이썬 smf.ols 함수에서 C(Unit)은 Unit을 각 Dummy Variable로 만들어준다.
  
- Unit Fixed Effect로 계산되는 방식은 다음과 같다.
  - $Unit_3$ 은 Control Group이기 때문에 구할 수 없다.
  - $Unit_1: (\frac{70 + 70}{2} - 50) = 20,\quad Unit_2: (\frac{30 + 50}{2} - 10) = 30$
  - $\frac{Unit_1 + Unit_2}{2}=\frac{20 + 30}{2} = 25$

##### Dummy Variable Example

In [14]:
import statsmodels.formula.api as smf
Panel['Gender'] = [1,1,1,0,0,0,1,1,1]
Panel['Age'] = [20,20,20,21,21,21,22,22,22]
Panel['Address'] = ["Area A","Area A","Area A","Area A","Area A","Area A","Area B","Area B","Area B"]

mod = smf.ols("Outcome ~ C(Unit) + Treatment", data=Panel).fit()
mod.summary().tables[1]

c:\Users\socsoft\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,46.6667,7.601,6.139,0.002,27.127,66.206
C(Unit)[T.2],-33.3333,7.601,-4.385,0.007,-52.873,-13.794
C(Unit)[T.3],-26.6667,9.309,-2.864,0.035,-50.597,-2.736
Treatment,25.0000,8.062,3.101,0.027,4.275,45.725


#### demeaned (Unit Fixed Effects)
- 하지만 Dummy Variable을 활용할 때, 차원의 문제가 발생하게 된다. 예를 들어, 백만명의 고객이 있는 데이터가 있다면 백만개의 Dummy Variable이 추가되기 때문에 계산이 엄청 느릴 것으로 예상해볼 수 있다.
  
- 따라서 Dummy Variable을 보완하기 위한 방법이 demeaned 방식이다.
  
- demeaned 으로 구하는 방식은 다음과 같다.
   - $(2)$의 $\delta \overline{D}_i + \overline{u}_i + \overline{\varepsilon}_{i}$ 에서 $\overline{u}_i$는 시간에 따라 변하지 않으므로 $\overline{u}_i=u_i$이다.
\begin{align}
   Y_{it}   & =\delta D_{it} + u_i + \varepsilon_{it}                                                             
   \\
   \overline{Y}_i            & =\delta \overline{D}_i + \overline{u}_i + \overline{\varepsilon}_{i}                                           
   \\
   (Y_{it} - \overline{Y}_i) & =(\delta D_{it} - \delta\overline{D}) + (u_i - u_i) + (\varepsilon_{it} - \overline{\varepsilon}_i)
   \\
   \ddot{Y}_{it}             & =\delta\ddot{D}_{it}+\ddot{\varepsilon}_{it}                                                        
\end{align}

- 이제 Unit Fixed Effects Model의 $\hat{\delta}$를 구해보자.
\begin{align*}
\big(\widehat{\delta}, \widehat{u}_1, \dots, \widehat{u}_N\big) = \underset{b,m_1,\dots,m_N}{\arg\min} \sum_{i=1}^N\sum_{t=1}^T (Y_{it}-D_{it}b- m_i)^2
\end{align*}
   - 위 식에서 First-Order Conditions(FOC, 최소화 하기 위한 첫번째 조건)는 다음과 같다.
\begin{align}
\sum\limits_{i=1}^N \sum\limits_{t=1}^T D_{it} \big(Y_{it} - D_{it}\widehat{\delta} - \widehat{u}i\big) &= 0 \
\text{and} \
\sum_{t=1}^T\big(Y_{it} - D_{it}\widehat{\delta} - \widehat{u}_i\big)&=0\\
\text{Therefore, for}\;i=1,\dots,N \\
\widehat{u}_i = \dfrac{1}{T} \sum_{t=1}^T\big(Y_{it}-D_{it}\widehat{\delta}\big)=& \overline{Y}_i - \overline{D}_i\widehat{\delta},\\
\text{where}\quad \overline{D}_i \equiv \dfrac{1}{T}\sum_{t=1}^TD_{it}; \bar{Y}_i &\equiv \dfrac{1}{T} \sum_{t=1}^T Y_{it}
\end{align}
   - $\widehat{u}_i = \overline{Y}_i - \overline{D}_i\widehat{\delta}$를 $\sum\limits_{i=1}^N \sum\limits_{t=1}^T D_{it} \big(Y_{it} - D_{it}\widehat{\delta} - \widehat{u}i\big) = 0$에 대입해보자.
\begin{align*}
& \sum\limits_{i=1}^N \sum\limits_{t=1}^T D_{it} \big(Y_{it} - D_{it}\widehat{\delta} - \widehat{u}i\big) \\
&= \sum_{i=1}^{N} \sum_{t=1}^{T}D_{it} (Y_{it} - D_{it}\widehat{\delta} - \overline{Y}_i + \overline{D}_i\widehat{\delta}) \\
&= \left( \sum_{i=1}^{N} \sum_{t=1}^{T} D_{it}(Y_{it} - \bar{Y}_i) \right) - \widehat{\delta}\left( \sum_{i=1}^{N} \sum_{t=1}^{T} D_{it}(D_{it} - \bar{D}_i) \right) = 0
\end{align*}
   - 즉, $\widehat{\delta}$는 다음과 같다.
\begin{align*}
   \widehat{\delta} = \frac{\sum\limits_{i=1}^{N} \sum\limits_{t=1}^{T}(D_{it} - \bar{D}_i)(D_{it} - \bar{D}_i)}{\sum\limits_{i=1}^{N} \sum\limits_{t=1}^{T}(Y_{it} - \bar{Y})(D_{it} - \bar{D}_i)} = \frac{\sum\limits_{i=1}^{N} \sum\limits_{t=1}^{T} \ddot{D}_{it}\ddot{Y}_{it}}{\sum\limits_{i=1}^{N} \sum\limits_{t=1}^{T} \ddot{D}_{it}^2}
\end{align*}

##### demeaned Example (Unit Fixed Effects)

In [ ]:
import numpy as np

Outcome_Unit = Panel.groupby('Unit')['Outcome'].transform('mean')

Treat_Unit = Panel.groupby('Unit')['Treatment'].transform('mean')

Treat_mean = np.mean(Panel['Treatment'])

Outcome_mean = np.mean(Panel['Outcome'])

Panel['dem_Treatment'] = Panel['Treatment'] - Treat_Unit + Treat_mean # Treatment

Panel['dem_Outcome'] = Panel['Outcome'] - Outcome_Unit + Outcome_mean # outcome

mod = smf.ols("dem_Outcome ~ dem_Treatment", data=Panel).fit()
mod.summary().tables[1]

### Two - Way Fixed Effects Model (Unit Fixed + Time Fixed)
- Unit Fixed Effects는 시간에 따라 변하는 변수들을 통제하지 못한다.
  - 예를 들어 인플레이션과 같은 변수들은 Unit Fixed Effects 방법으로 통제하지 못한다.
  
- 그렇기 때문에 Time Fixed를 고려한다면, 시간에 따라 변하는 변수들을 통제하여 Treatment Group과 Control Group과의 차이를 구할 수 있다.
  - 그 이유는 Time 관련 변수들은 시간별 각 그룹들의 값들이 모두 동일하다고 보기 때문에 Treatment Group과 Control Group의 차이를 구함으로써 Time에 대한 변수를 통제(상쇄)할 수 있다.
  
- 따라서 Unit 과 Time 변수를 통제하여 편향을 줄이고자 하는 방법이 Two - Way Fixed Effects Model 이다.

- Two - Way Fixed Effects Model의 식은 다음과 같다.
\begin{align}
  Y_{it} = \delta D_{it} + u_i + \gamma_t + \varepsilon_{it}
\end{align}

#### Dummy Variable Example (Two - Way Fixed Effects)
- 위 Panel Data를 활용하여 Unit 뿐만 아니라 Time까지 고려한 분석을 파이썬으로 실습해보자.
  
- Unit만 고려했을 때는 25의 값이 출력됐지만, Time까지 고려하면 40의 값으로 출력된다. 즉, 25의 값은 시간에 따라 변화하는 변수들을 고려하지 못했기 때문에 발생한 편향이라고 생각할 수 있다.
  
- Two - Way Fixed Effect로 계산되는 방식은 다음과 같다.
  - Time이 고려되었기 때문에 Treatment Group과 Control Group의 비교가 가능해진다.
  
  - $(Treatment_{After}(Treatment\;Group - Control\;Group) - Treatment_{Before}(Treatment\;Group - Control\;Group))$
  
  - $Unit_1$과 $Unit_2$의 비교는 모두 처치 변수이기 때문에 서로 상쇄 된다.
  
  - $Unit_1\;과\;Unit_3의\;비교: (Treatment_{After}(\frac{70+70}{2} - \frac{20+10}{2}) - Treatment_{Before}(50 - 30))=35$
  
  - $Unit_2\;과\;Unit_3의\;비교: (Treatment_{After}(\frac{30+50}{2} - \frac{20+10}{2}) - Treatment_{Before}(10 - 30))=45$
  
  - $\frac{35 + 45}{2} = 40$

In [42]:
mod = smf.ols("Outcome ~ C(Unit) + C(Time) + Treatment", data=Panel).fit()
mod.summary().tables[1]

c:\Users\socsoft\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,46.6667,7.698,6.062,0.009,22.168,71.165
C(Unit)[T.2],-33.3333,7.698,-4.330,0.023,-57.832,-8.835
C(Unit)[T.3],-16.6667,12.172,-1.369,0.264,-55.402,22.069
C(Time)[T.2],-16.6667,12.172,-1.369,0.264,-55.402,22.069
C(Time)[T.3],-13.3333,12.172,-1.095,0.353,-52.069,25.402
Treatment,40.0000,14.142,2.828,0.066,-5.007,85.007


#### demeaned Example (Two - Way Fixed Effects)
- Unit Fixed Effects와 동일하게 Two - Way Fixed Effects도 demeaned 방식으로 추정할 수 있다.

In [39]:
import numpy as np

Treat_Time = Panel.groupby('Time')['Treatment'].transform('mean')

Outcome_Time = Panel.groupby('Time')['Outcome'].transform('mean')

Outcome_Unit = Panel.groupby('Unit')['Outcome'].transform('mean')

Treat_Unit = Panel.groupby('Unit')['Treatment'].transform('mean')

Treat_mean = np.mean(Panel['Treatment'])

Outcome_mean = np.mean(Panel['Outcome'])

Panel['dem_Treatment'] = Panel['Treatment'] - Treat_Time - Treat_Unit + Treat_mean # Treatment

Panel['dem_Outcome'] = Panel['Outcome'] - Outcome_Time - Outcome_Unit + Outcome_mean # outcome

mod = smf.ols("dem_Outcome ~ dem_Treatment", data=Panel).fit()
mod.summary().tables[1]

c:\Users\socsoft\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-6.661e-16,2.057,-3.24e-16,1.000,-4.865,4.865
dem_Treatment,40.0000,9.258,4.320,0.003,18.108,61.892
